In [1]:
import os
import deepspeed
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [2]:
local_rank = int(os.getenv('LOCAL_RANK', '0'))
world_size = int(os.getenv('WORLD_SIZE', '8'))
world_size

8

In [3]:
model_path = "/data/yckj3980/tmp_model/bloomz-7b1-mt"
# max_memory_mapping = {6: "1GB", 7: "2GB"}
# model_8bit = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", load_in_8bit=True, max_memory=max_memory_mapping)
model_8bit = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# generation = pipeline("text-generation", model=model_8bit, tokenizer=tokenizer)

#prompt = "你是一个对话机器人，请用对话的方式回答下面问题。问题：你是谁？"
#inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#generated_ids = model_8bit.generate(**inputs)
#outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#outputs

In [4]:
ds_engine = deepspeed.init_inference(model_8bit,
                                 mp_size=4,
                                 dtype=torch.half,
                                 checkpoint=None,
                                 replace_with_kernel_inject=True)

[2023-04-13 11:16:19,628] [INFO] [logging.py:93:log_dist] [Rank -1] DeepSpeed info: version=0.8.3, git-hash=unknown, git-branch=unknown
[2023-04-13 11:16:19,633] [WARNING] [config_utils.py:75:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2023-04-13 11:16:19,634] [INFO] [logging.py:93:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
[2023-04-13 11:16:22,041] [INFO] [comm.py:634:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-04-13 11:16:22,234] [INFO] [comm.py:688:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.128.61.9, master_port=29500
[2023-04-13 11:16:22,237] [INFO] [comm.py:652:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


RuntimeError: the new group's world size should be less or equal to the world size set by init_process_group

In [ ]:
model = ds_engine.module
output = model('你好')
output

In [ ]:
import os
 
pid = os.getpid()
!kill -9 $pid